In [3]:
from data_utils import export_data_for_inspection
if __name__ == "__main__":

    print("--- 开始执行数据导出任务 ---")
    export_data_for_inspection(column_to_inspect='grossprofit_margin')

    print("\n" + "="*50 + "\n")
    print("--- 所有导出任务已完成 ---")

from data_utils import export_wide_panel

if __name__ == "__main__":
    print("--- 开始执行“长表转宽表”导出任务 ---")
    export_wide_panel(value_column='grossprofit_margin')
    print("\n" + "="*50 + "\n")
    print("--- 所有面板数据导出任务已完成 ---")


--- 开始执行数据导出任务 ---
--- 准备导出用于检查的 'grossprofit_margin' 原始数据 ---
数据加载成功！
数据已准备和排序完毕。

--- 正在导出数据到Excel文件 ---

[成功] 数据已成功导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/grossprofit_margin_source_data_for_inspection.xlsx
请打开此文件进行检查。


--- 所有导出任务已完成 ---
--- 开始执行“长表转宽表”导出任务 ---
--- 准备将 'grossprofit_margin' 数据转换为宽表 ---
数据加载成功！

去重前的数据行数: 169650
去重后的数据行数: 102305

数据清洗与去重完成。

正在执行 pivot_table 操作...
宽表创建成功！

--- 正在导出宽表到Excel文件 ---

[成功] 已将宽表格式的数据导出到: /Users/alan-hopiy/Documents/个人研究/基本面量化/grossprofit_margin_wide_format.xlsx


--- 所有面板数据导出任务已完成 ---


In [2]:
import pandas as pd
import numpy as np
import os # 引入os模块来处理文件路径

# ==============================================================================
#  步骤 0: 从您指定的Excel文件路径加载数据
# ==============================================================================
print("--- 步骤 0: 从Excel文件加载准备好的毛利率宽表数据 ---")

file_path = '/Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/初步数据/毛利率/stock_financial_indicators_grossprofit_margin_wide_panel.xlsx'

try:
    df_gross_margin = pd.read_excel(file_path, index_col=0)
    print(f"成功从路径 '{file_path}' 加载数据！")
    print("数据预览：")
    print(df_gross_margin.head())

except FileNotFoundError:
    print(f"[错误] 文件未找到！请检查路径是否正确: {file_path}")
    df_gross_margin = pd.DataFrame()
except Exception as e:
    print(f"读取Excel文件时出错: {e}")
    df_gross_margin = pd.DataFrame()

# ==============================================================================
#  数据预处理与因子计算流程
# ==============================================================================

if not df_gross_margin.empty:
    print("\n--- 开始执行数据预处理和因子计算 ---")
    
    # --- 步骤 1: 数据充分性筛选 ---
    print("\n--- 步骤 1: 数据充分性筛选 ---")
    valid_counts = df_gross_margin.notna().sum(axis=1)
    MIN_PERIODS = 12
    df_filtered = df_gross_margin[valid_counts >= MIN_PERIODS]
    print(f"原始公司数量: {len(df_gross_margin)}")
    print(f"筛选后剩余公司数量: {len(df_filtered)}")

    if not df_filtered.empty:
        # --- 步骤 2: 极端值处理 (缩尾) ---
        print("\n--- 步骤 2: 极端值处理 (缩尾) ---")
        lower_bound = df_filtered.stack().quantile(0.01)
        upper_bound = df_filtered.stack().quantile(0.99)
        df_winsorized = df_filtered.clip(lower=lower_bound, upper=upper_bound)
        print(f"已对数据在 [{lower_bound:.2f}, {upper_bound:.2f}] 范围内进行缩尾处理。")

        # --- 步骤 3: 计算处理后数据的平均值 ---
        print("\n--- 步骤 3: 计算处理后数据的平均值 ---")
        average_gm = df_winsorized.mean(axis=1)
        
        # --- 步骤 4: Z-Score 标准化 ---
        print("\n--- 步骤 4: Z-Score 标准化 ---")
        z_scores_gm = (average_gm - average_gm.mean()) / average_gm.std()
        print("Z-Score 计算完成。")

        # ==============================================================================
        #  新增部分：步骤 5 - 导出最终结果到Excel
        # ==============================================================================
        print("\n--- 步骤 5: 导出Z-score因子值到Excel文件 ---")
        
        # 定义输出的文件夹路径和完整文件路径
        output_dir = '/Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/z-scores'
        output_filename = 'Z值-毛利率-平均值.xlsx'
        output_path = os.path.join(output_dir, output_filename)
        
        try:
            # 确保输出文件夹存在
            os.makedirs(output_dir, exist_ok=True)
            
            # 将z_scores_gm这个Series转换为DataFrame并导出
            # z_scores_gm.to_frame()会把Series变成一列的DataFrame
            # 我们给这一列命名为 'z_score_gm'
            z_scores_gm.to_frame(name='z_score_gm').to_excel(output_path, index=True)
            
            print(f"\n[成功] 因子已成功导出到: {output_path}")
            
        except Exception as e:
            print(f"\n[失败] 文件导出失败: {e}")
        # ==============================================================================

    else:
        print("经过数据充分性筛选后，没有剩余的公司可供处理。")
else:
    print("数据加载失败或数据为空，未执行后续计算。")

--- 步骤 0: 从Excel文件加载准备好的毛利率宽表数据 ---
成功从路径 '/Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/初步数据/毛利率/stock_financial_indicators_grossprofit_margin_wide_panel.xlsx' 加载数据！
数据预览：
           2018-03-31  2018-06-30  2018-09-30  2018-12-31  2019-03-31  \
ts_code                                                                 
000002.SZ     34.1720     34.4372     34.7626     37.4816     35.0440   
000004.SZ     79.8477     81.9358     82.4253     81.8792     76.6223   
000006.SZ     22.9650     38.9920     38.9828     38.7866     39.2680   
000007.SZ     79.1646     73.3189     70.7013     69.3740     67.1958   
000008.SZ     57.7922     51.1164     48.5656     48.9184     53.0718   

           2019-06-30  2019-09-30  2019-12-31  2020-03-31  2020-06-30  ...  \
ts_code                                                                ...   
000002.SZ     36.2453     35.9903     36.2451     31.2754     31.8089  ...   
000004.SZ     75.9448     75.5123     73.0102     83.0721    

In [4]:
# file: build_gross_margin_growth_factor.py (修正版)

import pandas as pd
import numpy as np
import os

# 假设您的数据库连接信息在 config.py 文件中，我们需要用到其中的输出路径
try:
    import config
except ImportError:
    print("[警告] 无法找到配置文件 config.py。将使用当前目录作为输出目录。")
    # 定义一个默认的输出目录，以防config文件不存在
    class config:
        BASE_OUTPUT_DIR = '.'

# [核心修改] 使用新的、更稳健的成长率计算函数
def calculate_stable_growth(row, target_years, min_periods_in_window=4):
    """
    【升级版-稳健型】成长率计算函数，基于窗口平均值，以平滑异常值。

    - 近期值: 使用最近3个季度的平均值。
    - 基期值: 使用N年前基期及其前后3个季度（共7个季度）的平均值。
    - 智能基期: 如果基期窗口数据不足，则向更早的时期滑动窗口寻找有效数据。
    - 质量惩罚: 只有当近期平均值和基期平均值都为正时，才计算增长率，否则给予惩罚。
    """
    series = row.dropna()
    
    # --- 1. 计算近期平均值 (最近3个季度) ---
    if len(series) < 3:
        return np.nan # 数据太少，无法计算
    
    latest_window = series.iloc[-3:]
    latest_avg = latest_window.mean()

    # --- 2. 智能寻找并计算基期平均值 ---
    base_avg = np.nan
    actual_years = np.nan
    
    # 尝试从目标基期开始，向过去最多滑动5个季度来寻找一个有效窗口
    for shift in range(6): 
        base_period_center_idx = -(target_years * 4) - 1 - shift
        window_start_idx = base_period_center_idx - 3
        window_end_idx = base_period_center_idx + 4
        
        if window_start_idx >= -len(series) and window_end_idx <= 0:
            base_window = series.iloc[window_start_idx:window_end_idx]
            if base_window.count() >= min_periods_in_window:
                base_avg = base_window.mean()
                actual_years = (series.index.get_loc(latest_window.index[-1]) - series.index.get_loc(base_window.index[-1])) / 4.0
                break

    if pd.isna(base_avg):
        return np.nan

    # --- 3. 计算最终增长率或施加惩罚 ---
    if base_avg > 0 and latest_avg > 0:
        if actual_years < 1: 
            actual_years = 1.0
        return (latest_avg / base_avg) ** (1 / actual_years) - 1
    else:
        return -10

# ==============================================================================
#  步骤 1: 加载预处理好的毛利率面板数据 (不变)
# ==============================================================================
print("--- 步骤 1: 加载预处理好的毛利率面板数据 ---")
input_dir = config.BASE_OUTPUT_DIR
# 注意：请确保这个路径是正确的
input_path = "/Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/初步数据/毛利率/stock_financial_indicators_grossprofit_margin_wide_panel.xlsx"

try:
    gm_panel = pd.read_excel(input_path, index_col=0)
    print(f"成功加载文件，数据面板形状为: {gm_panel.shape}")
except FileNotFoundError:
    raise SystemExit(f"[错误] 输入文件未找到: {input_path}。请先运行之前的脚本生成此文件。")

# ==============================================================================
#  步骤 2: 计算多周期成长率 (带惩罚项)
# ==============================================================================
print("\n--- 步骤 2: 计算带质量惩罚的多周期毛利率成长率 ---")
results_df = pd.DataFrame(index=gm_panel.index)
for years in [3, 4, 5]:
    print(f"正在计算 {years}年期 稳健型CAGR...")
    col_name = f'gm_cagr_{years}yr_stable' # 建议改名以区分
    # [核心修改] 调用新的函数 calculate_stable_growth
    results_df[col_name] = gm_panel.apply(calculate_stable_growth, axis=1, target_years=years)

# ==============================================================================
#  后续步骤 3 到 8 的逻辑完全不变，只需对应修改列名即可
#  为了简洁，这里我将为您自动调整后续步骤的列名
# ==============================================================================

print("\n--- 步骤 3: 独立进行Z-score打分 ---")
zscore_cols = []
for years in [3, 4, 5]:
    cagr_col = f'gm_cagr_{years}yr_stable' # 使用新列名
    zscore_col = f'gm_zscore_{years}yr_stable' # 使用新列名
    zscore_cols.append(zscore_col)
    
    mean = results_df[cagr_col].mean()
    std = results_df[cagr_col].std()
    
    results_df[zscore_col] = (results_df[cagr_col] - mean) / std
    print(f"完成 {zscore_col} 的计算。")
    
# --- 步骤 4 到 8 (逻辑不变，仅为完整性展示) ---
print("\n--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---")
MIN_VALID_SCORES = 2
valid_score_counts = results_df[zscore_cols].count(axis=1)
results_df_filtered = results_df[valid_score_counts >= MIN_VALID_SCORES].copy()
print(f"筛选前公司数量: {len(results_df)}")
print(f"筛选后公司数量: {len(results_df_filtered)}")

print("\n--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---")
results_df_filtered[zscore_cols] = results_df_filtered[zscore_cols].fillna(0)

print("\n--- 步骤 6: 合成得分并进行最终标准化 ---")
composite_avg = results_df_filtered[zscore_cols].mean(axis=1)
final_mean = composite_avg.mean()
final_std = composite_avg.std()
final_score_col = 'composite_gm_growth_score_final_stable'
results_df_filtered[final_score_col] = (composite_avg - final_mean) / final_std

print("\n--- 步骤 7: 排序并预览最终结果 ---")
results_df_filtered.sort_values(by=final_score_col, ascending=False, inplace=True)
display_cols = [col for col in results_df_filtered.columns if 'cagr' not in col]
print(results_df_filtered[display_cols].head(10))

print("\n--- 步骤 8: 导出最终结果 ---")
output_dir = '/Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/z-scores'
output_filename = 'Z值-毛利率-增长率-稳健版.xlsx' # 建议改名以区分
output_path = os.path.join(output_dir, output_filename)

try:
    results_df_filtered.to_excel(output_path, index=True)
    print(f"\n[成功] 已将最终的毛利率成长分(稳健版)导出到: {output_path}")
except Exception as e:
    print(f"\n[失败] 文件导出失败: {e}")

[警告] 无法找到配置文件 config.py。将使用当前目录作为输出目录。
--- 步骤 1: 加载预处理好的毛利率面板数据 ---
成功加载文件，数据面板形状为: (5316, 29)

--- 步骤 2: 计算带质量惩罚的多周期毛利率成长率 ---
正在计算 3年期 CAGR...
正在计算 4年期 CAGR...
正在计算 5年期 CAGR...

--- 步骤 3: 独立进行Z-score打分 ---
完成 gm_zscore_3yr_penalized 的计算。
完成 gm_zscore_4yr_penalized 的计算。
完成 gm_zscore_5yr_penalized 的计算。

--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---
筛选前公司数量: 5316
筛选后公司数量: 5118

--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---
缺失Z-score填充完成。

--- 步骤 6: 合成得分并进行最终标准化 ---
最终得分合成与标准化完成。

--- 最终毛利率成长性打分排名预览 (前10名) ---
           gm_zscore_3yr_penalized  gm_zscore_4yr_penalized  \
ts_code                                                       
600156.SH                 3.186334                 0.284832   
600608.SH                 1.081144                 1.363405   
600251.SH                 2.444717                 0.286348   
603003.SH                 0.813351                 0.765388   
600506.SH                 0.249539                 1.297581   
600897.SH                 1.399149                 0.264375   
6003

In [11]:
# file: source/utils.py

import pandas as pd
import numpy as np
import os

def calculate_stable_growth(row, target_years, 
                            base_window_half_size=2, 
                            min_periods_in_window=3):
    """
    【可配置】成长率计算函数，基于窗口平均值，以平滑异常值。
    """
    series = row.dropna()
    
    # 1. 计算近期平均值 (固定为最近3季度)
    if len(series) < 3:
        return np.nan
    latest_window = series.iloc[-3:]
    latest_avg = latest_window.mean()

    # 2. 智能寻找并计算基期平均值
    base_avg = np.nan
    actual_years = np.nan
    
    for shift in range(6): 
        base_period_center_idx = -(target_years * 4) - 1 - shift
        window_start_idx = base_period_center_idx - base_window_half_size
        window_end_idx = base_period_center_idx + base_window_half_size + 1
        
        if window_start_idx >= -len(series) and window_end_idx <= 0:
            base_window = series.iloc[window_start_idx:window_end_idx]
            if base_window.count() >= min_periods_in_window:
                base_avg = base_window.mean()
                actual_years = (series.index.get_loc(latest_window.index[-1]) - series.index.get_loc(base_window.index[-1])) / 4.0
                break

    if pd.isna(base_avg):
        return np.nan

    # 3. 计算最终增长率或施加惩罚
    if base_avg > 0 and latest_avg > 0:
        if actual_years < 1: actual_years = 1.0
        return (latest_avg / base_avg) ** (1 / actual_years) - 1
    else:
        return -10

def build_composite_growth_factor(panel_data: pd.DataFrame, 
                                  factor_prefix: str,
                                  years_list: list = [3, 4, 5]):
    """
    【工作流函数】根据输入的宽表面板数据，构建一个复合成长因子。
    
    Args:
        panel_data (pd.DataFrame): 宽表数据，index为股票代码，columns为时间。
        factor_prefix (str): 因子前缀，用于命名，如 'gm' (毛利率) 或 'roe'。
        years_list (list): 要计算的成长年限列表。

    Returns:
        pd.DataFrame: 包含最终因子得分的DataFrame。
    """
    print(f"\n--- 开始构建复合成长因子: {factor_prefix}_growth ---")
    results_df = pd.DataFrame(index=panel_data.index)

    # 步骤 2: 计算多周期成长率
    print("\n--- 步骤 2: 计算稳健型CAGR ---")
    for years in years_list:
        col_name = f'{factor_prefix}_cagr_{years}yr_stable'
        results_df[col_name] = panel_data.apply(
            calculate_stable_growth, 
            axis=1, 
            target_years=years,
            base_window_half_size=2 # 使用5季度窗口作为平衡选择
        )

    # 步骤 3: 独立进行Z-score打分
    print("\n--- 步骤 3: 独立进行Z-score打分 ---")
    zscore_cols = []
    for years in years_list:
        cagr_col = f'{factor_prefix}_cagr_{years}yr_stable'
        zscore_col = f'{factor_prefix}_zscore_{years}yr_stable'
        zscore_cols.append(zscore_col)
        
        mean = results_df[cagr_col].mean()
        std = results_df[cagr_col].std()
        results_df[zscore_col] = (results_df[cagr_col] - mean) / std
    
    # 步骤 4 & 5: 筛选和填充
    print("\n--- 步骤 4 & 5: 筛选高质量公司并填充缺失值 ---")
    valid_score_counts = results_df[zscore_cols].count(axis=1)
    results_df_filtered = results_df[valid_score_counts >= 2].copy()
    results_df_filtered[zscore_cols] = results_df_filtered[zscore_cols].fillna(0)

    # 步骤 6: 合成并标准化最终得分
    print("\n--- 步骤 6: 合成得分并进行最终标准化 ---")
    composite_avg = results_df_filtered[zscore_cols].mean(axis=1)
    final_mean = composite_avg.mean()
    final_std = composite_avg.std()
    results_df_filtered[f'composite_{factor_prefix}_growth_score'] = (composite_avg - final_mean) / final_std
    
    # 步骤 7: 排序
    results_df_filtered.sort_values(by=f'composite_{factor_prefix}_growth_score', ascending=False, inplace=True)
    
    return results_df_filtered

import sys
import os
import pandas as pd

# --- 1. 环境与路径设置 ---
project_root = os.path.abspath('.') # 假设脚本在项目根目录运行
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录 '{project_root}' 已成功添加到Python搜索路径。")

import export_data_for_inspection, export_wide_panel
import build_composite_growth_factor
import source.config as config


# --- 2. 核心参数配置 ---
# 在这里定义您本次想要运行的任务
# ==============================================================================
# 要处理的数据库表名
TABLE_TO_LOAD = 'stock_financial_indicators'

# 要分析的具体指标列名 (必须与表中的列名一致)
METRIC_TO_ANALYZE = 'grossprofit_margin' 

# 用于生成列名和文件名的因子前缀 (建议使用简短的英文)
FACTOR_PREFIX = 'gm' # gm -> Gross Margin
# ==============================================================================


if __name__ == "__main__":
    
    print("====== 工作流开始 ======")
    
    # --- 步骤 A: 数据提取与准备 ---
    print(f"\n--- 步骤 A: 正在从表[{TABLE_TO_LOAD}]提取[{METRIC_TO_ANALYZE}]的数据 ---")
    export_wide_panel(
        table_name=TABLE_TO_LOAD,
        value_column=METRIC_TO_ANALYZE
    )

    # --- 步骤 B: 构建因子 ---
    # 首先，定位刚刚生成的宽表文件
    panel_file_name = f"{TABLE_TO_LOAD}_{METRIC_TO_ANALYZE}_wide_panel.xlsx"
    panel_file_path = os.path.join(config.BASE_OUTPUT_DIR, '初步数据', panel_file_name)

    print(f"\n--- 步骤 B: 正在从文件[{panel_file_name}]加载数据以构建因子 ---")
    try:
        panel_df = pd.read_excel(panel_file_path, index_col=0)
        print(f"成功加载文件，数据面板形状为: {panel_df.shape}")
    except FileNotFoundError:
        raise SystemExit(f"[错误] 输入文件未找到: {panel_file_path}。请确认步骤A是否成功执行。")

    # 调用我们封装好的因子构建函数
    final_factor_df = build_composite_growth_factor(
        panel_data=panel_df,
        factor_prefix=FACTOR_PREFIX
    )

    # --- 步骤 C: 保存最终因子得分 ---
    print("\n--- 步骤 C: 正在保存最终的因子得分 ---")
    output_dir = os.path.join(config.BASE_OUTPUT_DIR, 'z-scores')
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"Z值-{FACTOR_PREFIX}-复合增长因子.xlsx"
    output_path = os.path.join(output_dir, output_filename)

    try:
        final_factor_df.to_excel(output_path, index=True)
        print(f"\n[成功] 已将最终的因子得分导出到: {output_path}")
    except Exception as e:
        print(f"\n[失败] 最终文件导出失败: {e}")

    print("\n====== 工作流全部执行完毕！ ======")

ModuleNotFoundError: No module named 'export_data_for_inspection'

In [9]:
# file: run_workflow.py (新的主代码)

import sys
import os
import pandas as pd

# --- 1. 环境与路径设置 ---
project_root = os.path.abspath('.') # 假设脚本在项目根目录运行
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录 '{project_root}' 已成功添加到Python搜索路径。")

import export_data_for_inspection, export_wide_panel
import build_composite_growth_factor
import source.config as config


# --- 2. 核心参数配置 ---
# 在这里定义您本次想要运行的任务
# ==============================================================================
# 要处理的数据库表名
TABLE_TO_LOAD = 'stock_financial_indicators'

# 要分析的具体指标列名 (必须与表中的列名一致)
METRIC_TO_ANALYZE = 'grossprofit_margin' 

# 用于生成列名和文件名的因子前缀 (建议使用简短的英文)
FACTOR_PREFIX = 'gm' # gm -> Gross Margin
# ==============================================================================


if __name__ == "__main__":
    
    print("====== 工作流开始 ======")
    
    # --- 步骤 A: 数据提取与准备 ---
    print(f"\n--- 步骤 A: 正在从表[{TABLE_TO_LOAD}]提取[{METRIC_TO_ANALYZE}]的数据 ---")
    export_wide_panel(
        table_name=TABLE_TO_LOAD,
        value_column=METRIC_TO_ANALYZE
    )

    # --- 步骤 B: 构建因子 ---
    # 首先，定位刚刚生成的宽表文件
    panel_file_name = f"{TABLE_TO_LOAD}_{METRIC_TO_ANALYZE}_wide_panel.xlsx"
    panel_file_path = os.path.join(config.BASE_OUTPUT_DIR, '初步数据', panel_file_name)

    print(f"\n--- 步骤 B: 正在从文件[{panel_file_name}]加载数据以构建因子 ---")
    try:
        panel_df = pd.read_excel(panel_file_path, index_col=0)
        print(f"成功加载文件，数据面板形状为: {panel_df.shape}")
    except FileNotFoundError:
        raise SystemExit(f"[错误] 输入文件未找到: {panel_file_path}。请确认步骤A是否成功执行。")

    # 调用我们封装好的因子构建函数
    final_factor_df = build_composite_growth_factor(
        panel_data=panel_df,
        factor_prefix=FACTOR_PREFIX
    )

    # --- 步骤 C: 保存最终因子得分 ---
    print("\n--- 步骤 C: 正在保存最终的因子得分 ---")
    output_dir = os.path.join(config.BASE_OUTPUT_DIR, 'z-scores')
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"Z值-{FACTOR_PREFIX}-复合增长因子.xlsx"
    output_path = os.path.join(output_dir, output_filename)

    try:
        final_factor_df.to_excel(output_path, index=True)
        print(f"\n[成功] 已将最终的因子得分导出到: {output_path}")
    except Exception as e:
        print(f"\n[失败] 最终文件导出失败: {e}")

    print("\n====== 工作流全部执行完毕！ ======")

ModuleNotFoundError: No module named 'export_data_for_inspection'

In [12]:
# file: run_workflow.py (新的主代码)

import sys
import os
import pandas as pd

# --- 1. 环境与路径设置 ---
project_root = os.path.abspath('.') # 假设脚本在项目根目录运行
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录 '{project_root}' 已成功添加到Python搜索路径。")

from source.data_utils import export_data_for_inspection, export_wide_panel
from source.因子成长率计算 import build_composite_growth_factor
import source.config as config


# --- 2. 核心参数配置 ---
# 在这里定义您本次想要运行的任务
# ==============================================================================
# 要处理的数据库表名
TABLE_TO_LOAD = 'stock_financial_indicators'

# 要分析的具体指标列名 (必须与表中的列名一致)
METRIC_TO_ANALYZE = 'grossprofit_margin' 

# 用于生成列名和文件名的因子前缀 (建议使用简短的英文)
FACTOR_PREFIX = 'gm' # gm -> Gross Margin
# ==============================================================================


if __name__ == "__main__":
    
    print("====== 工作流开始 ======")
    
    # --- 步骤 A: 数据提取与准备 ---
    print(f"\n--- 步骤 A: 正在从表[{TABLE_TO_LOAD}]提取[{METRIC_TO_ANALYZE}]的数据 ---")
    export_wide_panel(
        table_name=TABLE_TO_LOAD,
        value_column=METRIC_TO_ANALYZE
    )

    # --- 步骤 B: 构建因子 ---
    # 首先，定位刚刚生成的宽表文件
    panel_file_name = f"{TABLE_TO_LOAD}_{METRIC_TO_ANALYZE}_wide_panel.xlsx"
    panel_file_path = os.path.join(config.BASE_OUTPUT_DIR, '初步数据', panel_file_name)

    print(f"\n--- 步骤 B: 正在从文件[{panel_file_name}]加载数据以构建因子 ---")
    try:
        panel_df = pd.read_excel(panel_file_path, index_col=0)
        print(f"成功加载文件，数据面板形状为: {panel_df.shape}")
    except FileNotFoundError:
        raise SystemExit(f"[错误] 输入文件未找到: {panel_file_path}。请确认步骤A是否成功执行。")

    # 调用我们封装好的因子构建函数
    final_factor_df = build_composite_growth_factor(
        panel_data=panel_df,
        factor_prefix=FACTOR_PREFIX
    )

    # --- 步骤 C: 保存最终因子得分 ---
    print("\n--- 步骤 C: 正在保存最终的因子得分 ---")
    output_dir = os.path.join(config.BASE_OUTPUT_DIR, 'z-scores')
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"Z值-{FACTOR_PREFIX}-复合增长因子.xlsx"
    output_path = os.path.join(output_dir, output_filename)

    try:
        final_factor_df.to_excel(output_path, index=True)
        print(f"\n[成功] 已将最终的因子得分导出到: {output_path}")
    except Exception as e:
        print(f"\n[失败] 最终文件导出失败: {e}")

    print("\n====== 工作流全部执行完毕！ ======")

ModuleNotFoundError: No module named 'source'

In [13]:
# file: run_workflow.py (请将此文件放在项目的根目录下, 即 quantitative-learning-journey/ 文件夹下)

import sys
import os
import pandas as pd

# --- 1. 环境与路径设置 (修正版) ---
# 将当前脚本所在的目录设置为项目根目录
# os.path.abspath('.') 会获取当前的工作目录
project_root = os.path.abspath('.') 
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录 '{project_root}' 已成功添加到Python搜索路径。")

# 现在，因为根目录已经在搜索路径中，下面的导入应该可以正常工作了
from source.data_utils import export_data_for_inspection, export_wide_panel
# 注意：我已将导入路径更新为您实际创建的文件名
from source.因子成长率计算 import build_composite_growth_factor 
import source.config as config


# --- 2. 核心参数配置 ---
# 在这里定义您本次想要运行的任务
# ==============================================================================
TABLE_TO_LOAD = 'stock_financial_indicators'
METRIC_TO_ANALYZE = 'grossprofit_margin' 
FACTOR_PREFIX = 'gm' # gm -> Gross Margin
# ==============================================================================


if __name__ == "__main__":
    
    print("====== 工作流开始 ======")
    
    # --- 步骤 A: 数据提取与准备 ---
    print(f"\n--- 步骤 A: 正在从表[{TABLE_TO_LOAD}]提取[{METRIC_TO_ANALYZE}]的数据 ---")
    export_wide_panel(
        table_name=TABLE_TO_LOAD,
        value_column=METRIC_TO_ANALYZE
    )

    # --- 步骤 B: 构建因子 ---
    panel_file_name = f"{TABLE_TO_LOAD}_{METRIC_TO_ANALYZE}_wide_panel.xlsx"
    panel_file_path = os.path.join(config.BASE_OUTPUT_DIR, '初步数据', panel_file_name)

    print(f"\n--- 步骤 B: 正在从文件[{panel_file_name}]加载数据以构建因子 ---")
    try:
        panel_df = pd.read_excel(panel_file_path, index_col=0)
        print(f"成功加载文件，数据面板形状为: {panel_df.shape}")
    except FileNotFoundError:
        raise SystemExit(f"[错误] 输入文件未找到: {panel_file_path}。请确认步骤A是否成功执行。")

    final_factor_df = build_composite_growth_factor(
        panel_data=panel_df,
        factor_prefix=FACTOR_PREFIX
    )

    # --- 步骤 C: 保存最终因子得分 ---
    print("\n--- 步骤 C: 正在保存最终的因子得分 ---")
    output_dir = os.path.join(config.BASE_OUTPUT_DIR, 'z-scores')
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"Z值-{FACTOR_PREFIX}-复合增长因子.xlsx"
    output_path = os.path.join(output_dir, output_filename)

    try:
        final_factor_df.to_excel(output_path, index=True)
        print(f"\n[成功] 已将最终的因子得分导出到: {output_path}")
    except Exception as e:
        print(f"\n[失败] 最终文件导出失败: {e}")

    print("\n====== 工作流全部执行完毕！ ======")

ModuleNotFoundError: No module named 'source'

In [1]:
# file: run_workflow.py (最终版)
# 您可以将此文件保存在项目根目录，或者在Jupyter Notebook中运行此代码块

import sys
import os
import pandas as pd

# --- 1. 环境与路径设置 (最终修正版 - 更健壮的路径查找) ---
def find_project_root(sentinel_name='source'):
    """自动向上查找，直到找到包含特定文件/文件夹（如'source'）的项目根目录。"""
    current_path = os.path.abspath('.')
    while True:
        if sentinel_name in os.listdir(current_path):
            return current_path
        parent_path = os.path.dirname(current_path)
        # 如果已经到达文件系统的根目录，则停止
        if parent_path == current_path:
            raise FileNotFoundError(f"无法找到包含 '{sentinel_name}' 的项目根目录。请确保您在项目内部运行。")
        current_path = parent_path

try:
    project_root = find_project_root()
    if project_root not in sys.path:
        sys.path.append(project_root)
        print(f"项目根目录 '{project_root}' 已成功添加到Python搜索路径。")
except FileNotFoundError as e:
    print(e)
    # 如果找不到，可以提供一个备用路径，或者直接退出
    sys.exit(1)


# --- 导入您自己的模块 ---
# 注意：我已将导入路径更新为您实际创建的文件名
from source.data_utils import export_wide_panel
from source.因子成长率计算 import build_composite_growth_factor 
import source.config as config


# --- 2. 核心参数配置 ---
# ==============================================================================
TABLE_TO_LOAD = 'stock_financial_indicators'
METRIC_TO_ANALYZE = 'grossprofit_margin' 
FACTOR_PREFIX = 'gm' # gm -> Gross Margin
# ==============================================================================


if __name__ == "__main__":
    
    print("\n====== 工作流开始 ======")
    
    # --- 步骤 A: 数据提取与准备 ---
    print(f"\n--- 步骤 A: 正在从表[{TABLE_TO_LOAD}]提取[{METRIC_TO_ANALYZE}]的数据 ---")
    export_wide_panel(
        table_name=TABLE_TO_LOAD,
        value_column=METRIC_TO_ANALYZE
    )

    # --- 步骤 B: 构建因子 ---
    panel_file_name = f"{TABLE_TO_LOAD}_{METRIC_TO_ANALYZE}_wide_panel.xlsx"
    panel_file_path = os.path.join(config.BASE_OUTPUT_DIR, '初步数据', panel_file_name)

    print(f"\n--- 步骤 B: 正在从文件[{panel_file_name}]加载数据以构建因子 ---")
    try:
        panel_df = pd.read_excel(panel_file_path, index_col=0)
        print(f"成功加载文件，数据面板形状为: {panel_df.shape}")
    except FileNotFoundError:
        raise SystemExit(f"[错误] 输入文件未找到: {panel_file_path}。请确认步骤A是否成功执行。")

    final_factor_df = build_composite_growth_factor(
        panel_data=panel_df,
        factor_prefix=FACTOR_PREFIX
    )

    # --- 步骤 C: 保存最终因子得分 ---
    print("\n--- 步骤 C: 正在保存最终的因子得分 ---")
    output_dir = os.path.join(config.BASE_OUTPUT_DIR, 'z-scores')
    os.makedirs(output_dir, exist_ok=True)
    
    output_filename = f"Z值-{FACTOR_PREFIX}-复合增长因子.xlsx"
    output_path = os.path.join(output_dir, output_filename)

    try:
        final_factor_df.to_excel(output_path, index=True)
        print(f"\n[成功] 已将最终的因子得分导出到: {output_path}")
    except Exception as e:
        print(f"\n[失败] 最终文件导出失败: {e}")

    print("\n====== 工作流全部执行完毕！ ======")

项目根目录 '/Users/alan-hopiy/Documents/quantitative-learning-journey' 已成功添加到Python搜索路径。

====== 工作流开始 ======

--- 步骤 A: 正在从表[stock_financial_indicators]提取[grossprofit_margin]的数据 ---
--- 准备将表[stock_financial_indicators]的 'grossprofit_margin' 数据转换为宽表 ---
长表数据加载成功！

数据清洗前总行数: 249561
因包含空值被删除的行数: 5604
因重复被删除的行数: 102689
数据清洗后剩余行数: 141268

正在执行 pivot_table 操作...
宽表创建成功！

[成功] 宽表数据已成功导出到: /Users/alan-hopiy/Documents/quantitative-learning-journey/outputs/初步数据/初步数据/stock_financial_indicators_grossprofit_margin_wide_panel.xlsx

--- 步骤 B: 正在从文件[stock_financial_indicators_grossprofit_margin_wide_panel.xlsx]加载数据以构建因子 ---
成功加载文件，数据面板形状为: (5316, 29)

--- 开始构建复合成长因子: gm_growth ---

--- 步骤 2: 计算带质量惩罚的多周期成长率 ---
正在计算 3年期 CAGR...
正在计算 4年期 CAGR...
正在计算 5年期 CAGR...

--- 步骤 3: 独立进行Z-score打分 ---
完成 gm_zscore_3yr_stable 的计算。
完成 gm_zscore_4yr_stable 的计算。
完成 gm_zscore_5yr_stable 的计算。

--- 步骤 4: 基于有效得分数量，筛选高质量公司 ---
筛选前公司数量: 5316
筛选后公司数量: 4994

--- 步骤 5: 对缺失的Z-score填充0值 (作为中性惩罚) ---
缺失Z-score填充完成。

--- 步骤 6: 合成得分并进行